In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

In [ ]:
train_df = pd.read_csv('/kaggle/input/llm-classification-finetuning/train.csv')
test_df = pd.read_csv('/kaggle/input/llm-classification-finetuning/test.csv')
sample_submission = pd.read_csv('/kaggle/input/llm-classification-finetuning/sample_submission.csv')

train_df.head()


In [ ]:
def get_winner(row):
    if row['winner_model_a'] == 1:
        return 'A'
    elif row['winner_model_b'] == 1:
        return 'B'
    else:
        return 'tie'

train_df['label'] = train_df.apply(get_winner, axis=1)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(data=train_df, x='label')
plt.title("Distribution of Winner Labels (A vs B vs Tie)")
plt.show()


In [ ]:
def get_winner(row):
    if row['winner_model_a'] == 1:
        return 0  # model_a
    elif row['winner_model_b'] == 1:
        return 1  # model_b
    else:
        return 2  # tie

train_df['label'] = train_df.apply(get_winner, axis=1)


X_train, X_val, y_train, y_val = train_test_split(train_df['prompt'], train_df['label'], test_size=0.1, random_state=42)

# Tokenization
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

 

# Convert to Dataset format
from datasets import Dataset

# تحويل إلى تنسيق Dataset المطلوب
train_dataset = Dataset.from_pandas(pd.DataFrame({'text': X_train, 'label': y_train}))
val_dataset = Dataset.from_pandas(pd.DataFrame({'text': X_val, 'label': y_val}))


def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset = val_dataset.map(tokenize, batched=True)



In [ ]:
!pip install -U transformers


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

from transformers import TrainingArguments

 


trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()


In [ ]:
#import pandas as pd

# Load test data from competition input
#test_df = pd.read_csv('/kaggle/input/llm-classification-finetuning/test.csv')

# Generate dummy predictions (always 'model_a')
#preds = ['model_a'] * len(test_df)

# Prepare submission file
#submission = pd.DataFrame({
#    'id': test_df['id'],
#    'winner': preds
#})

# Save the submission file
#submission.to_csv('submission.csv', index=False)


In [ ]:
import pandas as pd

# Create dummy predictions or use your actual model outputs
submission = pd.DataFrame({
    'id': test_df['id'],
    'winner': ['model_a'] * len(test_df)  # Replace with real predictions if available
})

# Save the submission file
submission.to_csv('submission.csv', index=False)
print("✅ submission.csv saved.")
